In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from plotnine import *

In [18]:
import warnings
warnings.filterwarnings("ignore")

Final model which used is XGBoost and before that was Elastic net.

In [19]:
df = pd.read_csv('/Users/benji/Documents/Machine Learning/gsb-544-fall-2024-house-prices/train_new.csv')

In [20]:
df.head()

,SalePrice,PID,Lot Frontage,Lot Area,Street,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,...,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Functional,Screen Porch,Pool Area,Yr Sold,Sale Type
0,159000,531363010,80.0,9605,Pave,SawyerW,1Fam,1Story,7,6,...,1,1,3,6,1218,Typ,0,0,2009,WD
1,271900,906203120,90.0,14684,Pave,SawyerW,1Fam,1Story,7,7,...,2,0,3,7,2196,Typ,0,0,2009,WD
2,137500,916176030,NaN,14375,Pave,Timber,1Fam,SLvl,6,6,...,1,0,3,7,1344,Typ,233,0,2009,COD
3,248500,528180130,48.0,6472,Pave,NridgHt,TwnhsE,1Story,9,5,...,2,0,2,6,1456,Typ,0,0,2009,WD
4,167000,528290030,61.0,9734,Pave,Gilbert,1Fam,SLvl,7,5,...,2,1,3,7,1374,Typ,0,0,2009,WD


In [21]:
df.columns

Index(['SalePrice', 'PID', 'Lot Frontage', 'Lot Area', 'Street',
       'Neighborhood', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Roof Style', 'Heating', 'Central Air',
       'Electrical', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'TotRms AbvGrd', 'Gr Liv Area', 'Functional', 'Screen Porch',
       'Pool Area', 'Yr Sold', 'Sale Type'],
      dtype='object')

In [22]:
df.isna().sum()

SalePrice          0
PID                0
Lot Frontage     362
Lot Area           0
Street             0
Neighborhood       0
Bldg Type          0
House Style        0
Overall Qual       0
Overall Cond       0
Year Built         0
Roof Style         0
Heating            0
Central Air        0
Electrical         1
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
TotRms AbvGrd      0
Gr Liv Area        0
Functional         0
Screen Porch       0
Pool Area          0
Yr Sold            0
Sale Type          0
dtype: int64

In [23]:
# print("Correlation of numeric features to SalesPrice")
# corr = df.corr()
# corr.sort_values(["SalePrice"], ascending = False, inplace = True)
# print(corr.SalePrice)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2197 entries, 0 to 2196
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SalePrice      2197 non-null   int64  
 1   PID            2197 non-null   int64  
 2   Lot Frontage   1835 non-null   float64
 3   Lot Area       2197 non-null   int64  
 4   Street         2197 non-null   object 
 5   Neighborhood   2197 non-null   object 
 6   Bldg Type      2197 non-null   object 
 7   House Style    2197 non-null   object 
 8   Overall Qual   2197 non-null   int64  
 9   Overall Cond   2197 non-null   int64  
 10  Year Built     2197 non-null   int64  
 11  Roof Style     2197 non-null   object 
 12  Heating        2197 non-null   object 
 13  Central Air    2197 non-null   object 
 14  Electrical     2196 non-null   object 
 15  Full Bath      2197 non-null   int64  
 16  Half Bath      2197 non-null   int64  
 17  Bedroom AbvGr  2197 non-null   int64  
 18  TotRms A

In [25]:
#df['Lot Frontage'].fillna(df['Lot Frontage'].mean(),inplace=True)
df.fillna({'Lot Frontage': df['Lot Frontage'].mean()}, inplace=True)
df.isna().sum()
df.dropna(inplace=True)

In [26]:
df.head()

,SalePrice,PID,Lot Frontage,Lot Area,Street,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,...,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Functional,Screen Porch,Pool Area,Yr Sold,Sale Type
0,159000,531363010,80.000000,9605,Pave,SawyerW,1Fam,1Story,7,6,...,1,1,3,6,1218,Typ,0,0,2009,WD
1,271900,906203120,90.000000,14684,Pave,SawyerW,1Fam,1Story,7,7,...,2,0,3,7,2196,Typ,0,0,2009,WD
2,137500,916176030,69.549319,14375,Pave,Timber,1Fam,SLvl,6,6,...,1,0,3,7,1344,Typ,233,0,2009,COD
3,248500,528180130,48.000000,6472,Pave,NridgHt,TwnhsE,1Story,9,5,...,2,0,2,6,1456,Typ,0,0,2009,WD
4,167000,528290030,61.000000,9734,Pave,Gilbert,1Fam,SLvl,7,5,...,2,1,3,7,1374,Typ,0,0,2009,WD


In [27]:
df = df[df['Neighborhood'] != 'Landmrk']
df['Neighborhood'].unique()


array(['SawyerW', 'Timber', 'NridgHt', 'Gilbert', 'NAmes', 'Mitchel',
       'NWAmes', 'MeadowV', 'CollgCr', 'OldTown', 'BrkSide', 'Sawyer',
       'StoneBr', 'NoRidge', 'Edwards', 'ClearCr', 'Somerst', 'IDOTRR',
       'Crawfor', 'SWISU', 'Blmngtn', 'Veenker', 'NPkVill', 'BrDale',
       'Greens', 'Blueste', 'GrnHill'], dtype=object)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2195 entries, 0 to 2196
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SalePrice      2195 non-null   int64  
 1   PID            2195 non-null   int64  
 2   Lot Frontage   2195 non-null   float64
 3   Lot Area       2195 non-null   int64  
 4   Street         2195 non-null   object 
 5   Neighborhood   2195 non-null   object 
 6   Bldg Type      2195 non-null   object 
 7   House Style    2195 non-null   object 
 8   Overall Qual   2195 non-null   int64  
 9   Overall Cond   2195 non-null   int64  
 10  Year Built     2195 non-null   int64  
 11  Roof Style     2195 non-null   object 
 12  Heating        2195 non-null   object 
 13  Central Air    2195 non-null   object 
 14  Electrical     2195 non-null   object 
 15  Full Bath      2195 non-null   int64  
 16  Half Bath      2195 non-null   int64  
 17  Bedroom AbvGr  2195 non-null   int64  
 18  TotRms AbvGrd

In [29]:
X = df.drop(['PID','SalePrice'],axis=1)
y = np.log(df['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.25, random_state=42)                  

In [30]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
# from sklearn.linear_model import ElasticNet
# from sklearn.model_selection import StratifiedKFold, train_test_split
# from sklearn.compose import ColumnTransformer, make_column_selector

# catVar = make_column_selector(dtype_include=['object'])

# # Cross-validation for the model
# def modelPipeline(modelName, X=X_train, y=y_train):
#     # Define the column transformer to standardize and encode categorical features
#     ct = ColumnTransformer(
#         transformers=[
#             ('standardize', StandardScaler(), make_column_selector(dtype_include= ['int','float64'])),  # Adjust this column as needed
#             ('encode', OneHotEncoder(drop='first', sparse_output=False), catVar),  # Specify categorical columns
#         ], 
#         remainder='passthrough'  # Keep other columns as they are
#     )
    
#     # Create the full pipeline with column transformation and model fitting
#     pipeline = Pipeline(
#         steps=[
#             ('columnTransform', ct),
#             ('Model', modelName)
#         ]
#     )
    
#     # Perform cross-validation (cv=10 for 10-fold cross-validation)
    
#     cross_val_scores = cross_val_score(pipeline, X, y, cv=10, scoring='neg_mean_squared_error')  # Use MSE for regression
    
#     # Calculate and print the average cross-validation score
#     mean_cv_score = cross_val_scores.mean()
#     print("Cross-validation MSE:", mean_cv_score)

#     return pipeline, mean_cv_score

# # Hyperparameter tuning with cross-validation
# def tunning(model, X, y, paramGrid=None):
#     if paramGrid is None:
#         paramGrid = {
#         'Model__alpha': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength (alpha)
#         'Model__l1_ratio': [0.1, 0.5, 0.9, 1],           # ElasticNet mixing parameter (balance between L1 and L2)
#         'Model__max_iter': [100, 200, 300],              # Number of iterations for convergence
#     }

#     # Create a pipeline with column transformation and model fitting for grid search
#     ct = ColumnTransformer(
#         transformers=[
#             ('standardize', StandardScaler(), make_column_selector(dtype_include= ['int','float64'])),  # Adjust this column as needed
#             ('encode', OneHotEncoder(drop = 'first',sparse_output=False,handle_unknown='ignore'), make_column_selector(dtype_include=['object'])),  # Specify categorical columns
#         ], 
#         remainder='passthrough'
#     )
    
#     pipeline = Pipeline(
#         steps=[
#             ('columnTransform', ct),
#             ('Model', model)
#         ]
#     )
    
#     # Perform grid search with cross-validation (cv=10 for 10-fold cross-validation)
#     gridSearch = GridSearchCV(pipeline, param_grid=paramGrid, scoring='neg_mean_squared_error', cv=10)  # Use MSE for regression
#     gridSearch.fit(X, y)
    
#     return gridSearch.best_params_

# # Example of using the pipeline and tuning function
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# elasticnet = ElasticNet()
# model, mse = modelPipeline(elasticnet, X_train, y_train)  # Now uses cross-validation
# best_params = tunning(elasticnet, X, y)
# print("Best parameters:", best_params)


In [31]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.impute import SimpleImputer
import numpy as np

catVar = make_column_selector(dtype_include=['object'])

# Cross-validation for the model
def modelPipeline(modelName, X=X_train, y=y_train):
    # Define the column transformer to standardize and encode categorical features
    ct = ColumnTransformer(
        transformers=[
            ('standardize', StandardScaler(), make_column_selector(dtype_include=['int', 'float64'])),
            ('encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), catVar),
        ],
        remainder='passthrough'  # Keep other columns as they are
    )
    
    # Create the full pipeline with column transformation and model fitting
    pipeline = Pipeline(
        steps=[
            ('columnTransform', ct),
            ('Model', modelName)
        ]
    )
    pipeline.fit(X,y)
    # Perform cross-validation (cv=10 for 10-fold cross-validation)
    cv = KFold(n_splits=10, shuffle=True, random_state=42)
    cross_val_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_mean_squared_error')
    
    # Calculate and print the average cross-validation RMSE
    mean_cv_score = cross_val_scores.mean()
    print("Cross-validation RMSE:", np.sqrt(-mean_cv_score))

    return pipeline, mean_cv_score

# Hyperparameter tuning with cross-validation
def tunning(model, X, y, paramGrid=None):
    if paramGrid is None:
        paramGrid = {
            'Model__alpha': [0.001, 0.01, 0.1, 1, 10, 100],
            'Model__l1_ratio': [0.1, 0.5, 0.9, 1],
            'Model__max_iter': [100, 200, 300],
        }

    # Create a pipeline with column transformation and model fitting for grid search
    ct = ColumnTransformer(
        transformers=[
            ('cat', Pipeline([
                ('impute', SimpleImputer(strategy='most_frequent')),
                ('encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
            ]), catVar),
            
            ('num', StandardScaler(), make_column_selector(dtype_include=['int64', 'float64'])),
        ],
        remainder='passthrough'  # Keep other columns as they are
    )
    
    pipeline = Pipeline(
        steps=[
            ('columnTransform', ct),
            ('Model', model)
        ]
    )
    
    # Perform grid search with cross-validation (cv=10 for 10-fold cross-validation)
    gridSearch = GridSearchCV(pipeline, param_grid=paramGrid, scoring='neg_mean_squared_error', cv=10)
    gridSearch.fit(X, y)
    
    return gridSearch.best_params_

# Example of using the pipeline and tuning function
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
elasticnet = ElasticNet(max_iter=10000)
model, mse = modelPipeline(elasticnet, X_train, y_train)  # Now uses cross-validation
best_params = tunning(elasticnet, X, y)
print("Best parameters:", best_params)


Cross-validation RMSE: 0.41439887552380816
Best parameters: {'Model__alpha': 0.001, 'Model__l1_ratio': 0.1, 'Model__max_iter': 100}


In [32]:
best_en = modelPipeline(ElasticNet(alpha = .001, l1_ratio= 0.1,max_iter=100))

Cross-validation RMSE: 0.1407474238189505


In [33]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.impute import SimpleImputer
import numpy as np

catVar = make_column_selector(dtype_include=['object'])

# Cross-validation for the model
def modelPipeline(modelName, X, y):
    # Define the column transformer to handle both categorical and numerical features
    ct = ColumnTransformer(
        transformers=[
            # Impute and OneHotEncode for categorical columns
            ('cat', Pipeline([
                ('impute', SimpleImputer(strategy='most_frequent')),
                ('encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
            ]), catVar),
            
            # Standardize numerical columns
            ('num', StandardScaler(), make_column_selector(dtype_include=['int64', 'float64'])),
        ],
        remainder='passthrough'  # Keep other columns as they are
    )
    
    # Create the full pipeline with column transformation and model fitting
    pipeline = Pipeline(
        steps=[
            ('columnTransform', ct),
            ('Model', modelName)
        ]
    )
    pipeline.fit(X, y)
    
    # Perform cross-validation (cv=10 for 10-fold cross-validation)
    cv = KFold(n_splits=10, shuffle=True, random_state=42)
    cross_val_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_mean_squared_error')
    
    # Calculate and print the average cross-validation RMSE
    mean_cv_score = cross_val_scores.mean()
    print("Cross-validation RMSE:", np.sqrt(-mean_cv_score))

    return pipeline, mean_cv_score

# Hyperparameter tuning with cross-validation
def tunning(model, X, y, paramGrid=None):
    if paramGrid is None:
        paramGrid = {
            'Model__alpha': [0.001, 0.01, 0.1, 1, 10, 100],
            'Model__max_iter': [100, 200, 300],
        }

    # Create a pipeline with column transformation and model fitting for grid search
    ct = ColumnTransformer(
        transformers=[
            ('cat', Pipeline([
                ('impute', SimpleImputer(strategy='most_frequent')),
                ('encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
            ]), catVar),
            
            ('num', StandardScaler(), make_column_selector(dtype_include=['int64', 'float64'])),
        ],
        remainder='passthrough'  # Keep other columns as they are
    )
    
    pipeline = Pipeline(
        steps=[
            ('columnTransform', ct),
            ('Model', model)
        ]
    )
    
    # Perform grid search with cross-validation (cv=10 for 10-fold cross-validation)
    gridSearch = GridSearchCV(pipeline, param_grid=paramGrid, scoring='neg_mean_squared_error', cv=10)
    gridSearch.fit(X, y)
    
    return gridSearch.best_params_

# Example of using the pipeline and tuning function
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
ridge = Ridge(max_iter=10000)
model, mse = modelPipeline(ridge, X, y)  # Now using Ridge
best_params = tunning(ridge, X, y)
print("Best parameters:", best_params)


Cross-validation RMSE: 0.1500123392163481
Best parameters: {'Model__alpha': 10, 'Model__max_iter': 100}


In [34]:
best_ridge = modelPipeline(Ridge(alpha = 10,max_iter=100),X,y)

Cross-validation RMSE: 0.14976453842157197


In [35]:
best_en = modelPipeline(ElasticNet(alpha = .001, l1_ratio= 0.1,max_iter=100),X,y)

Cross-validation RMSE: 0.1496245462413447


In [36]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.impute import SimpleImputer
import numpy as np

# catVar = make_column_selector(dtype_include=['object'])

# # Cross-validation for the model
# def modelPipeline(modelName, X, y):
#     # Define the column transformer to handle both categorical and numerical features
#     ct = ColumnTransformer(
#         transformers=[
#             # Impute and OneHotEncode for categorical columns
#             ('cat', Pipeline([
#                 ('impute', SimpleImputer(strategy='most_frequent')),
#                 ('encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
#             ]), catVar),
            
#             # Standardize numerical columns
#             ('num', StandardScaler(), make_column_selector(dtype_include=['int64', 'float64'])),
#         ],
#         remainder='passthrough'  # Keep other columns as they are
#     )
    
#     # Create the full pipeline with column transformation and model fitting
#     pipeline = Pipeline(
#         steps=[
#             ('columnTransform', ct),
#             ('Model', modelName)
#         ]
#     )
#     pipeline.fit(X, y)
#     # Perform cross-validation (cv=10 for 10-fold cross-validation)
#     cv = KFold(n_splits=10, shuffle=True, random_state=42)
#     cross_val_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_mean_squared_error')
    
#     # Calculate and print the average cross-validation RMSE
#     mean_cv_score = cross_val_scores.mean()
#     print("Cross-validation RMSE:", np.sqrt(-mean_cv_score))

#     return pipeline, mean_cv_score

# # Hyperparameter tuning with cross-validation
# def tunning(model, X, y, paramGrid=None):
#     if paramGrid is None:
#         paramGrid = {
#             'Model__n_estimators': [50, 100, 200],
#             'Model__learning_rate': [0.01, 0.1, 0.3],
#             'Model__max_depth': [3, 5, 7],
#             'Model__subsample': [0.8, 1.0],
#             'Model__colsample_bytree': [0.8, 1.0],
#         }

#     # Create a pipeline with column transformation and model fitting for grid search
#     ct = ColumnTransformer(
#         transformers=[
#             ('cat', Pipeline([
#                 ('impute', SimpleImputer(strategy='most_frequent')),
#                 ('encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
#             ]), catVar),
            
#             ('num', StandardScaler(), make_column_selector(dtype_include=['int64', 'float64'])),
#         ],
#         remainder='passthrough'  # Keep other columns as they are
#     )
    
#     pipeline = Pipeline(
#         steps=[
#             ('columnTransform', ct),
#             ('Model', model)
#         ]
#     )
    
#     # Perform grid search with cross-validation (cv=10 for 10-fold cross-validation)
#     gridSearch = GridSearchCV(pipeline, param_grid=paramGrid, scoring='neg_mean_squared_error', cv=10)
#     gridSearch.fit(X, y)
    
#     return gridSearch.best_params_

# Example of using the pipeline and tuning function
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# xgb_model = XGBRegressor(objective='reg:squarederror', n_jobs=-1, random_state=42)
# model, mse = modelPipeline(xgb_model, X_train, y_train)  # Now using XGBoost Regressor
# best_params = tunning(xgb_model, X, y)
# print("Best parameters:", best_params)


In [38]:
best_xg = modelPipeline(XGBRegressor(
                                    colsample_bytree = 0.8, 
                                    learning_rate= 0.1, 
                                     max_depth= 5, 
                                     n_estimators= 200,
                                     subsample= 0.8
),X,y)

Cross-validation RMSE: 0.13200515389260983


In [ ]:
test = pd.read_csv('/Users/benji/Documents/Machine Learning/gsb-544-fall-2024-house-prices/test_new.csv')
test.head()

,PID,Lot Frontage,Lot Area,Street,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,...,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Functional,Screen Porch,Pool Area,Yr Sold,Sale Type
0,907135180,60,8070,Pave,CollgCr,1Fam,1Story,4,5,1994,...,1,0,3,5,990,Typ,0,0,2007,WD
1,528181040,40,6792,Pave,NridgHt,TwnhsE,1Story,7,5,2005,...,2,0,2,6,1368,Typ,0,0,2006,New
2,528175010,44,6371,Pave,NridgHt,TwnhsE,1Story,7,5,2009,...,2,0,2,6,1358,Typ,0,0,2010,New
3,531379030,70,8304,Pave,SawyerW,1Fam,2Story,6,5,1997,...,2,1,3,7,1837,Typ,0,0,2006,WD
4,923275090,37,6951,Pave,Mitchel,1Fam,1Story,5,5,1984,...,1,0,3,5,923,Typ,0,0,2008,WD


In [ ]:
demo = test.drop(['PID'],axis =1)

In [ ]:
# SaleDataModel, rsme_random = modelPipeline(ElasticNet(alpha = .001, l1_ratio= 0.1,max_iter=100),X,y)
best_xg, ran_rsme = modelPipeline(XGBRegressor(
                                    colsample_bytree = 0.8, 
                                    learning_rate= 0.1, 
                                     max_depth= 5, 
                                     n_estimators= 200,
                                     subsample= 0.8
),X,y)

# SaleData = best_xg.predict(demo)

#SaleDataModel, rsme_random = modelPipeline(Ridge(alpha=10, max_iter=100), X, y)
#SaleData = SaleDataModel.predict(demo)
SaleData = best_xg.predict(demo)

Cross-validation RMSE: 0.13200515389260983


In [ ]:
# Generate predictions for the test data
final_predictions = pd.DataFrame(
    {
        "PID": test['PID'],  # Extracting id_num from test data
        "SalePrice": np.expm1(SaleData)  # Making predictions on the test data
    }
)

# Save the predictions to a CSV file
final_predictions.to_csv('SalesData.csv', index=False)

# Output the first few rows to confirm
print(final_predictions.head())

         PID      SalePrice
0  907135180  132412.125000
1  528181040  201585.343750
2  528175010  212428.218750
3  531379030  193427.406250
4  923275090  122729.101562


ChatGPT was used for debuggeg and understanding some parts of the code.